#### 문제 4-2 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용)


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print(TAVILY_API_KEY[:4])

In [3]:
import warnings
warnings.filterwarnings("ignore")

from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_core.documents import Document

from langchain_openai import ChatOpenAI
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage

# LangGraph MessagesState라는 미리 만들어진 상태를 사용
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import create_react_agent

from textwrap import dedent
from typing import List, Literal, Tuple
from pydantic import BaseModel, Field

import gradio as gr

from pprint import pprint

import uuid

#from IPython.display import Image, display

In [4]:
embeddings_model = UpstageEmbeddings(model="solar-embedding-1-large")

# cafe_db 벡터 저장소 로드 (4-2는 카페 메뉴 전용)
cafe_db = FAISS.load_local(
    "../db/cafe_db", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# Tool 정의 (조건부 분기 추가)
from langchain.agents import tool
import re

@tool
def search_cafe(query: str) -> List[str]:
    """
    카페 메뉴에서 정보를 검색합니다.
    사용자 질문을 분류하여 (메뉴/가격/추천) 맞춤 검색을 실행합니다.
    """

    # 1) 문의 유형 분류
    q_lower = query.lower()
    if "가격" in query or "얼마" in query:
        query_type = "price"
    elif "추천" in query or "인기" in query:
        query_type = "recommend"
    else:
        query_type = "menu"

    # 2) 유형별 검색 전략
    if query_type == "price":
        docs = cafe_db.similarity_search("카페 메뉴 가격", k=5)
    elif query_type == "recommend":
        docs = cafe_db.similarity_search(query, k=3)
        if not docs:
            docs = cafe_db.similarity_search("인기 카페 메뉴", k=3)
    else:  # 일반 메뉴 문의
        docs = cafe_db.similarity_search(query, k=4)

    # 3) 결과 정리
    if not docs:
        return ["관련 카페 메뉴 정보를 찾을 수 없습니다."]

    formatted_docs = [
        f'<Document source="{doc.metadata.get("source","N/A")}">\n{doc.page_content}\n</Document>'
        for doc in docs
    ]
    return formatted_docs

` LangChain 내장 도구`
- 일반 웹 검색을 위한 Tavily 초기화

In [5]:
# LangChain 내장 Tavily 도구 (웹 검색)
from langchain.agents import tool
from langchain_community.tools import TavilySearchResults

@tool
def search_web(query: str) -> List[str]:
    """
    카페 메뉴 DB(cafe_db)에 없는 정보나 최신 정보를 검색합니다.
    예: '올해 가장 인기 있는 음료 트렌드', '카페 신메뉴 소식' 등
    """
    # Tavily 검색 엔진 초기화
    tavily_search = TavilySearchResults(max_results=3)
    docs = tavily_search.invoke(query)

    # 검색 결과 포맷팅
    formatted_docs = "\n\n---\n\n".join(
        [
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in docs
        ]
    )

    if len(docs) > 0:
        return formatted_docs
    
    return "관련 최신 정보를 찾을 수 없습니다."

### bind_tools() 함수로 LLM과 Tool 연결하기

In [6]:
from langchain_openai import ChatOpenAI

# LLM 모델 설정
# llm = ChatOpenAI(model="gpt-4o-mini", streaming=True)  # OpenAI 사용시
llm = ChatUpstage(
    model="solar-pro",
    base_url="https://api.upstage.ai/v1",
    temperature=0.5
)
print(llm.model_name)

# 도구 목록 (카페 전용 DB 검색 + 웹 검색)
tools = [search_cafe, search_web]

# 모델에 도구를 바인딩 (RunnableBindings)
llm_with_tools = llm.bind_tools(tools=tools)

solar-pro


In [ ]:
# === 도구 호출 테스트 ===

# 1) 카페 메뉴 DB 검색 테스트
tool_call = llm_with_tools.invoke(
    [HumanMessage(content="아메리카노 가격은 얼마인가요?")]
)
print("\n[카페 메뉴 DB 호출 결과]")
pprint(tool_call.additional_kwargs)

# 2) 웹 검색(Tavily) 테스트
tool_call = llm_with_tools.invoke(
    [HumanMessage(content="최근에 공개된 오픈소스 LLM 모델은 어떤 것들이 있나요?")]
)
print("\n[웹 검색 호출 결과]")
pprint(tool_call.additional_kwargs)

# 3) 단순 연산 (도구 불필요) 테스트
tool_call = llm_with_tools.invoke(
    [HumanMessage(content="3+3은 얼마인가요?")]
)
print("\n[일반 질의 결과]")
pprint(tool_call.additional_kwargs)

# 전체 응답 객체 확인 (마지막 호출 기준)
print("\n[최종 tool_call 객체]")
pprint(tool_call)

In [8]:
# === 도구 노드 정의 ===
tools = [search_cafe, search_web]
tool_node = ToolNode(tools=tools)

# === 1) 카페 메뉴 DB 검색 테스트 ===
tool_call = llm_with_tools.invoke(
    [HumanMessage(content="아메리카노 가격은 얼마인가요?")]
)
print("\n[카페 메뉴 질의 → LLM의 tool_calls]")
pprint(tool_call.additional_kwargs)

# ToolNode 실행 (실제 search_cafe 동작)
results = tool_node.invoke({"messages": [tool_call]})
print("\n[ToolNode 실행 결과]")
for result in results['messages']:
    print(type(result))
    print(result.content)
    print('**** --------------------------- ****')

# === 2) 웹 검색(Tavily) 테스트 ===
tool_call = llm_with_tools.invoke(
    [HumanMessage(content="최근에 공개된 오픈소스 LLM 모델은 어떤 것들이 있나요?")]
)
print("\n[웹 검색 질의 → LLM의 tool_calls]")
pprint(tool_call.additional_kwargs)

results = tool_node.invoke({"messages": [tool_call]})
print("\n[ToolNode 실행 결과]")
for result in results['messages']:
    print(result.content)
    print()


[카페 메뉴 질의 → LLM의 tool_calls]
{'refusal': None,
 'tool_calls': [{'function': {'arguments': '{"query": '
                                           '"\\uc544\\uba54\\ub9ac\\uce74\\ub178 '
                                           '\\uac00\\uaca9"}',
                              'name': 'search_cafe'},
                 'id': 'chatcmpl-tool-44bb2373ba9845d0a5fdb68c52d269cf',
                 'type': 'function'}]}

[ToolNode 실행 결과]
<class 'langchain_core.messages.tool.ToolMessage'>
["<Document source=\"../data/cafe_menu_data.txt\">\n4. 바닐라 라떼\n   • 가격: ₩6,000\n   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽\n   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.\n</Document>", "<Document source=\"../data/cafe_menu_data.txt\">\n2. 카페라떼\n   • 가격: ₩5,500\n   • 주요 원료: 에스프레소, 스팀 밀크\n   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 대표적인 밀크 커피입니다. 크리미한 질감과 부드러운 맛이 특징이며, 다양한 시럽과 토핑 추가가 가능합니다. 라떼 아트로 시각적 즐거움도 제공합니다.\n</Document>", "<Document source=\"../data/cafe_menu_data.t

C:\Users\user\AppData\Local\Temp\ipykernel_17420\2015373739.py:12: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=3)



[ToolNode 실행 결과]
<Document href="https://brunch.co.kr/@sparta/88"/>
brunch 매거진 AI가 서말이라도 꿰어야 보배 LLM 오픈소스는 누구나 무료로 LLM을 수정하고 활용할 수 있도록 시중에 공유되어 있는데요. 2024년 이후에 출시된 오픈소스 LLM 중 뛰어난 성능을 갖춘 모델을 소개해 드립니다. LLaMA 3은 2024년 4월 메타 AI가 공개한 오픈소스 LLM입니다. 하지만 Llama 3은 GPT-4와 달리 누구에게나 열려있는 ‘오픈소스’라는 점에서 유의미합니다. 오픈소스 LLM 중 뛰어난 성능을 보이는 Llama는 챗봇, 텍스트 번역 및 요약 등 다양한 분야에서 활용할 수 있습니다. 이미지와 같은 시각적 입력을 텍스트로 변환하여 출력하는 모델인 ‘Falcon 2 11B VLM(vision-to-language model)’도 출시 예정입니다. Falcon 2 11B 모델은 출시된 5월, 허깅 페이스에서 52,000개 이상의 다운로드 수를 기록했습니다. Gemma는 2024년 2월 구글 AI가 출시한 오픈소스 LLM입니다. * 오픈소스 "누구나 큰일 낼 수 있어" 누구나 큰일 낼 수 있도록, 모두를 위한 소프트웨어 교육을 만들어갑니다. brunch membership
</Document>

---

<Document href="https://www.elastic.co/kr/blog/open-source-llms-guide"/>
콘텐츠 생성, 번역, 분류 및 기타 다양한 사용 사례와 같은 다양한 자연어 처리(NLP) 작업을 수행하도록 훈련할 수 있는 신경망 아키텍처를 기반으로 구축되었습니다. LLM은 또한 데이터 처리 및 분석 방법을 확장하여 클라우드 보안, 검색, Observability를 향상시키는 데 중요한 역할을 할 수 있습니다. 오픈 소스 LLM을 사용하면 모든 개인이나 기업이 라이선스 비용을 지불하지 않고도 원하는 대로 LLM을 사용할 수 있습니다. GPT-NeoX-20B는 주로 연구 목적

## ReAct Agent


In [10]:
from langgraph.prebuilt import create_react_agent

# === 1) 도구 목록 (카페 DB + 웹 검색) ===
tools = [search_cafe, search_web]

# === 2) ReAct Agent 생성 ===
agent = create_react_agent(
    llm, 
    tools=tools,
)

# === 3) 카페 메뉴 질의 테스트 ===
inputs = {"messages": [HumanMessage(content="아메리카노 가격은 얼마인가요?")]}
messages = agent.invoke(inputs)

print("\n[카페 메뉴 질의 응답]")
for m in messages['messages']:
    m.pretty_print()

# === 4) 웹 검색 질의 테스트 ===
inputs = {"messages": [HumanMessage(content="최근에 공개된 오픈소스 LLM 모델은 어떤 것들이 있나요?")]}
messages = agent.invoke(inputs)

print("\n[웹 검색 질의 응답]")
for m in messages['messages']:
    m.pretty_print()


[카페 메뉴 질의 응답]
================================ Human Message =================================

아메리카노 가격은 얼마인가요?
================================== Ai Message ==================================

[아메리카노 가격을 확인하기 위해 `search_cafe` 함수를 호출하는 것이 필수적입니다. 이 함수는 카페 메뉴 데이터베이스에서 가격 정보를 직접 검색하므로, 질문에 대한 정확한 답변을 제공할 수 있습니다. 웹 검색(`search_web`)은 최신 정보보다는 DB 내 기본 가격 조회에 불필요합니다.]
Tool Calls:
  search_cafe (chatcmpl-tool-a621e6b59d28464c868dca4d840df1f9)
 Call ID: chatcmpl-tool-a621e6b59d28464c868dca4d840df1f9
  Args:
    query: 아메리카노 가격
================================= Tool Message =================================
Name: search_cafe

["<Document source=\"../data/cafe_menu_data.txt\">\n4. 바닐라 라떼\n   • 가격: ₩6,000\n   • 주요 원료: 에스프레소, 스팀 밀크, 바닐라 시럽\n   • 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.\n</Document>", "<Document source=\"../data/cafe_menu_data.txt\">\n2. 카페라떼\n   • 가격: ₩5,500\n   • 주요 원료: 에스프레소, 스팀 밀크\n   • 설명: 진한 에스프레소에 부드럽게 스팀한 우유를 넣어 만든 